# Script to extract SRTM elevations over study sites through Google Earth Engine (GEE)

Adapted from [2022 ICESat Hackweek notebook](https://github.com/ICESAT-2HackWeek/website2022/blob/main/book/tutorials/DataVisualization/Visualization_Earth_Engine_geemap_IS2_HW_2022.ipynb)

In [1]:
import os
import ee
import pandas as pd
import geopandas as gpd
import geemap
import numpy as np
from IPython.display import Image
import matplotlib.pyplot as plt
import h5py
from rasterio import plot, warp

In [2]:
# -----GEE Authentication and Initialization
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

### AOI #1: Grand Mesa, CO

In [3]:
# -----Create geojson AOI
# used geojson.io
AOI1 = ee.Geometry({
        "type": "Polygon",
        "coordinates": [
          [
            [
              -108.24005126953124,
              38.884619201291905
            ],
            [
              -107.9,
              38.884619201291905
            ],
            [
              -107.9,
              39.11727568585598
            ],
            [
              -108.24005126953124,
              39.11727568585598
            ],
            [
              -108.24005126953124,
              38.884619201291905
            ]
          ]
        ]
      })

# -----Query for SRTM clip to AOI
SRTM1 = ee.Image("USGS/SRTMGL1_003").clip(AOI1)

# -----Plot elevations
# from: https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api
SRTM1_url = SRTM1.getThumbUrl({
    'min': 1000, 'max': 5000,
    'palette': ['white', 'green']})
Image(url=SRTM1_url)

### AOI #2: Mores Creek Summit, ID

In [4]:
# -----Create geojson AOI
# used geojson.io
AOI2 = ee.Geometry({
        "type": "Polygon",
        "coordinates": [
          [
            [
              -115.74131011962889,
              43.8855215890078
            ],
            [
              -115.61874389648438,
              43.8855215890078
            ],
            [
              -115.61874389648438,
              43.963661859597536
            ],
            [
              -115.74131011962889,
              43.963661859597536
            ],
            [
              -115.74131011962889,
              43.8855215890078
            ]
          ]
        ]
      })

# -----Query for SRTM clip to AOI
SRTM2 = ee.Image("USGS/SRTMGL1_003").clip(AOI2)

# -----Plot elevations
# from: https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api
SRTM2_url = SRTM2.getThumbUrl({
    'min': 1000, 'max': 5000,
    'palette': ['white', 'green']})
Image(url=SRTM2_url)

In [7]:
# -----Create a map
Map = geemap.Map()
Map.addLayer(AOI1)
Map.addLayer(AOI2)
Map.addLayer(SRTM2, {min:1000, max:3000})
Map.addLayer(SRTM1, {min:1000, max:3000})
Map.setCenter(-112.179660, 40.936019, zoom=6)
Map

Map(center=[40.936019, -112.17966], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

In [40]:
# -----Practice sampling... 
# Load some ICESat-2 data, for now we load one track, one beam (GT2R/L)
data_path = '/home/jovyan/data/'
is2_file = 'processed_ATL08_20200920013522_13240806_005_01.h5' # Sept = snow-off 

with h5py.File(data_path+is2_file, 'r') as f:
    is2_gt2l = pd.DataFrame(data={'lat': f['gt2l/land_segments/latitude'][:],
                                 'lon': f['gt2l/land_segments/longitude'][:],
                                  'h_te_best_fit': f['gt2l/land_segments/terrain/h_te_best_fit'][:]})

# select one of the beams and convert it to a geodataframe
gf = gpd.GeoDataFrame(is2_gt2l, geometry=gpd.points_from_xy(is2_gt2l.lon, is2_gt2l.lat), crs='epsg:7661')

(array('d', [-115.66415405273438]), array('d', [43.937278747558594]))

In [46]:
# -----function to convert pandas GeoDataFrame to ee.FeatureCollection
from functools import reduce
def feature2ee(gdf):

    g = [i for i in gdf.geometry]
    features=[]
    #for Point geo data type
    if (gdf.geom_type[0] == 'Point'):
        for i in range(len(g)):
            g = [i for i in gdf.geometry]
            x,y = g[i].coords.xy
            cords = np.dstack((x,y)).tolist()
            double_list = reduce(lambda x,y: x+y, cords)
            single_list = reduce(lambda x,y: x+y, double_list)
               
            g=ee.Geometry.Point(single_list)
            feature = ee.Feature(g)
            features.append(feature)

        ee_object = ee.FeatureCollection(features)
        return ee_object
        
# convert SRTM ee.Image to pd.DataFrame
is_ee = feature2ee(gf)
is_ee.getInfo()

{'type': 'FeatureCollection',
 'columns': {'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-115.66415405273438, 43.937278747558594]},
   'id': '0',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-115.6642837524414, 43.93638229370117]},
   'id': '1',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-115.6644058227539, 43.93548583984375]},
   'id': '2',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-115.6645278930664, 43.93458557128906]},
   'id': '3',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-115.66464233398438, 43.93368911743164]},
   'id': '4',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-115.66474914550781, 43.93279266357422]},
   'id': '5',
   'properties': {}},
  

In [57]:
# -----sample SRTM at features
SRTM1_sample = SRTM1.sampleRegions(is_ee)
SRTM1_sample